# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [40]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr
import hvplot.xarray
import geopandas as gpd

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [41]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hobart,-42.8794,147.3294,29.107,79,75,8.75,AU,1675316209
1,1,huarmey,-10.0681,-78.1522,29.483,82,100,2.56,PE,1675316209
2,2,katsuura,35.1333,140.3000,28.157,35,100,3.45,JP,1675316210
3,3,charlestown,-32.9500,151.6667,30.603,51,14,1.79,AU,1675315977
4,4,bluff,-46.6000,168.3333,29.317,79,92,1.89,NZ,1675316210


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [49]:
#%%capture --no-display


# Configure the map plot
map_humidity = city_data_df.hvplot.points("Lng","Lat", geo = True, titles = "OSM", size = "Humidity", color="City", frame_height = 450, tiles = True)


# Display the map
map_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [53]:
# Narrow down cities that fit criteria and drop any results with null values
indexCriteria = city_data_df[ (city_data_df["Max Temp"] > 27) | (city_data_df['Max Temp'] < 21) | (city_data_df['Wind Speed'] > 4.5) | (city_data_df['Cloudiness'] > 0)  ].index

# Drop any rows with null values
new_city = city_data_df.drop(indexCriteria)

# Display sample data
new_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,qaanaaq,77.4840,-69.3632,25.186,56,0,4.12,GL,1675316215
31,31,hay river,60.8156,-115.7999,24.114,75,0,2.06,CA,1675316219
63,63,yellowknife,62.4560,-114.3525,23.568,67,0,1.54,CA,1675316230
311,311,weymouth,42.2209,-70.9398,26.913,63,0,0.89,US,1675316317
362,362,petropavlovsk-kamchatskiy,53.0452,158.6483,26.248,57,0,2.00,RU,1675316334
416,416,broome,42.2506,-75.8330,26.593,74,0,2.57,US,1675316352
467,467,storforshei,66.4043,14.5233,25.907,84,0,0.00,NO,1675316369
482,482,matagami,49.7502,-77.6328,24.621,83,0,2.06,CA,1675316374
539,539,tongliao,43.6125,122.2653,26.904,19,0,3.89,CN,1675316392


### Step 3: Create a new DataFrame called `hotel_df`.

In [51]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = new_city.copy()
hotel_df = hotel_df.drop(columns=["Wind Speed", "Date","Max Temp","Cloudiness"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Humidity,Country,Hotel Name
18,18,qaanaaq,77.4840,-69.3632,56,GL,
31,31,hay river,60.8156,-115.7999,75,CA,
63,63,yellowknife,62.4560,-114.3525,67,CA,
311,311,weymouth,42.2209,-70.9398,63,US,
362,362,petropavlovsk-kamchatskiy,53.0452,158.6483,57,RU,
416,416,broome,42.2506,-75.8330,74,US,
467,467,storforshei,66.4043,14.5233,84,NO,
482,482,matagami,49.7502,-77.6328,83,CA,
539,539,tongliao,43.6125,122.2653,19,CN,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set parameters to search for a hotel
params = {
    "radius": 10000,
    "types": "lodging",
    "key": geoapify_key}   

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"{lat},{lng}"
    params["bias"] = f"{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

SyntaxError: invalid syntax (268015281.py, line 3)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)